In [ ]:
!apt-get update

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [347 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,327 kB]
Get

In [ ]:
!apt-get install -y xvfb ffmpeg freeglut3-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.15).
The following additional packages will be installed:
  freeglut3 libegl-dev libgl-dev libgl1-mesa-dev libgles-dev libgles1
  libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev libglx-dev
  libopengl-dev libxt-dev
Suggested packages:
  libxt-doc
The following NEW packages will be installed:
  freeglut3 freeglut3-dev libegl-dev libgl-dev libgl1-mesa-dev libgles-dev
  libgles1 libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev
  libglx-dev libopengl-dev libxt-dev
0 upgraded, 14 newly installed, 0 to remove and 41 not upgraded.
Need to get 1,192 kB of archives.
After this operation, 6,439 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 freeglut3 amd64 2.8.1-6 [74.0 kB]
Get:2 http://archive.ub

In [ ]:
!pip install gymnasium pybullet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 MB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pybullet: filename=pybullet-3.2.7-cp312-cp312-linux_x86_64.whl size=99873155 sha256=aa31e6a215e99ff8f543183638c470150be4f9add0807b6903aa2fbd5fb8b013
  Stored in directory: /root/.cache/pip/wheels/72/95/1d/b336e5ee612ae9a019bfff4dc0bedd100ee6f0570db205fdf8
Successfully built pybullet


In [ ]:
!git clone https://github.com/axelbr/racecar_gym.git

Cloning into 'racecar_gym'...
remote: Enumerating objects: 2693, done.
remote: Counting objects: 100% (343/343), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 2693 (delta 291), reused 254 (delta 254), pack-reused 2350 (from 1)
Receiving objects: 100% (2693/2693), 35.30 MiB | 24.16 MiB/s, done.
Resolving deltas: 100% (1614/1614), done.


In [ ]:
%cd racecar_gym

/content/racecar_gym


In [ ]:
!pip install -e .

Obtaining file:///content/racecar_gym
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 MB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 105.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
!pip install pyvirtualdisplay


In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
# --- 0) Headless display for Colab ---
!apt-get update -qq
!apt-get install -y -qq xvfb ffmpeg freeglut3-dev
!pip -q install gymnasium pybullet imageio pyvirtualdisplay
!git clone -q https://github.com/axelbr/racecar_gym.git || true
%cd racecar_gym
!pip -q install -e .

from pyvirtualdisplay import Display
_display = Display(visible=0, size=(1400, 900))
_display.start()

# --- 1) Imports ---
import gymnasium as gym
import imageio.v2 as imageio   # use v2 API to avoid deprecation hiccups
import numpy as np

# IMPORTANT: registers racecar env IDs like "SingleAgentAustria-v0"
import racecar_gym.envs.gym_api

# --- 2) Create env (rgb_array mode returns images) ---
env = gym.make("SingleAgentAustria-v0", render_mode="rgb_array")
obs, info = env.reset(seed=0)

frames = []

def try_get_frame_from_obs(observation):
    """
    racecar_gym often returns a dict per agent/sensor.
    Try common keys to extract an RGB frame if env.render() returns None.
    """
    # Single-agent env: observation may be a dict of sensors
    # Try camera-like keys
    candidate_keys = ["rgb", "rgb_camera", "camera", "front_camera", "image"]
    if isinstance(observation, dict):
        # some envs return {'A': {...sensors...}}; unwrap once
        if len(observation) == 1 and isinstance(list(observation.values())[0], dict):
            observation = list(observation.values())[0]
        for k in candidate_keys:
            if k in observation:
                arr = observation[k]
                arr = np.asarray(arr)
                if arr.ndim == 3 and arr.shape[-1] in (3, 4):
                    return arr
    return None

# --- 3) Rollout & record ---
for _ in range(200):
    action = env.action_space.sample()
    obs, reward, term, trunc, info = env.step(action)

    # Preferred: ask the env to render a frame in rgb_array mode
    frame = None
    try:
        frame = env.render()
    except Exception:
        frame = None

    # Fallback: some scenarios expose the camera directly in obs dict
    if frame is None:
        frame = try_get_frame_from_obs(obs)

    # Only append if we actually have an image
    if frame is not None:
        frames.append(frame)

    if term or trunc:
        obs, info = env.reset()

env.close()

# --- 4) Save & display GIF ---
if len(frames) == 0:
    print("No frames were captured. This can happen on the first run if tracks are downloading. Run the cell again.")
else:
    imageio.mimsave('racecar.gif', frames, fps=30)
    from IPython.display import Image, display
    display(Image(filename='racecar.gif'))
